# [Module 4.4] Personalize 캠페인과 실시간 상호 작용 실험 하기

이 노트북은 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.
이 노트북은 다음과 같은 작업을 합니다.
- 이벤트 추적기 생성 
    - 이전 노트북에서 사용한 이벤트 추적기를 사용합니다. 현재 하드 코딩되어 있습니다. 직접 수정해서 사용하세요.
- user_id를 할당 합니다.
- user_id 로 추천을 받습니다. 
- 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 두 번째, 세 번째, 네 번째 추천 결과 보기

In [261]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
from random import randint

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [262]:
%store -r


In [263]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 추적기 생성

이전 노트북에서 생성한 것을 사용 합니다. 
**퍼스널 라이즈 콘솔에서 이벤트 추적기로 이동 하신 후에 TRACKING_ID 를 카피 하세요.**

In [264]:
TRACKING_ID = '50d0f462-527d-4cb4-9252-1e77c9476cf1' # 반드시 수정이 필요 합니다.

#### help 함수(영화 타이틀, 장르 리턴)



In [265]:
# First load items into memory
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()
items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
print(items.tail())

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

def get_movie_genre(movie_id):
    """
    Takes in an ID, returns a genre
    """
    movie_id = int(movie_id)
    movie_genre=items[items['ITEM_ID']==movie_id]['GENRE']
    return (movie_genre.tolist())

# Help 함수
def rec_item_list(item_list):
    recommendation_title_list = []
    recommendation_genre_list = []
    # recommendation_id_list=[]
    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    
        
    recommendations_df = pd.DataFrame(data={'OriginalRecs':recommendation_title_list,
                                            'Genre': recommendation_genre_list})
    return recommendations_df                                                



      ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


## 특정 유저 선택

테스트하고자 하는 user_id 의 번호를 기입 하세요.

In [266]:
# user_id = 4375 # 특정 유저 선택, 결과 확인 후에 추후에 주석 표시하여 랜더 유저 선택할 수 있습니다.
user_id = 3100 # 특정 유저 선택, 결과 확인 후에 추후에 주석 표시하여 랜더 유저 선택할 수 있습니다.
print("USER: {}".format(user_id))



USER: 3100


## 선택된 유저의 과거 영화 레이팅 정보
훈련에 사용된 영화 상호 작용 데이타를 확인 합니다. 
- 여기서 **장르** 를 확인 하세요. 그리고 추천 받은 것이 비슷한 장르인지 확인 하십시오.

In [267]:
from utils import get_rich_dataset

# 학습에 사용된 warm_train 데이터 셋 로딩
df_warm_train = pd.read_csv(warm_train_interaction_filename)
# item 정보 로딩
item_meta = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)

def get_recentVeiw(user_id, df_rich, num_history):
    '''
    최신 리스트 및 추천 아이템을 제공 함
    '''
    history_items = df_rich[df_rich['USER_ID']==user_id].tail(num_history)    
    return history_items


# 영황 타이틀, 장르를 포함한 정보 리턴
df_warm_train_rich = get_rich_dataset(df_warm_train, item_meta)
df_warm_train_rich = df_warm_train_rich.sort_values('TIMESTAMP').copy()
# df_warm_train_rich

user_history = get_recentVeiw(user_id, df_warm_train_rich, num_history=20)
user_history





/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
18065,3100,2762,"Sixth Sense, The (1999)",Thriller,969591386,2000-09-22 02:56:26
257219,3100,628,Primal Fear (1996),Drama|Thriller,969591430,2000-09-22 02:57:10
259003,3100,2391,"Simple Plan, A (1998)",Crime|Thriller,969591448,2000-09-22 02:57:28
337427,3100,3102,Jagged Edge (1985),Thriller,969591524,2000-09-22 02:58:44
44616,3100,1834,"Spanish Prisoner, The (1997)",Drama|Thriller,969591524,2000-09-22 02:58:44
235716,3100,3101,Fatal Attraction (1987),Thriller,969591579,2000-09-22 02:59:39
147565,3100,230,Dolores Claiborne (1994),Drama|Thriller,969591609,2000-09-22 03:00:09
252791,3100,1608,Air Force One (1997),Action|Thriller,969591641,2000-09-22 03:00:41
286725,3100,1407,Scream (1996),Horror|Thriller,969591718,2000-09-22 03:01:58
249600,3100,3005,"Bone Collector, The (1999)",Thriller,969591786,2000-09-22 03:03:06


## 이벤트 트래커에 업데이트 없이 첫번째 추천을 받습니다.
- 이전의 과거 히스토리의 결과와 비교 해보세요
- `user_personalization_campaign_arn` 는 이전 노트북에서 변수로 저장이 되어 있습니다. `%store -r`를 통해서 변수를 로딩 합니다. 하지만 여기서 테스트 목적으로 다른 캠페인 ARN을 가져왔습니다. 
    - **[중요] 주석을 하시거나 반드시 아래 ARN을 수정 하세요.** 


In [268]:
# 이전 노트북에서 사용한 것을 사용하려면 주석 표시를 하시고, 다른 캠페인을 하시려면 바꾸어 주세요.
user_personalization_campaign_arn = 'arn:aws:personalize:ap-northeast-2:057716757052:campaign/Movielens-user-personalization-campaign-60498'


In [269]:

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,    
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

rec_exp_df = rec_item_list(item_list)
rec_exp_df

Recommendations for user:  3100


,OriginalRecs,Genre
0,"[War Room, The (1993)]",[Documentary]
1,[When We Were Kings (1996)],[Documentary]
2,[Paris Is Burning (1990)],[Documentary]
3,[Wild Man Blues (1998)],[Documentary]
4,"[Celluloid Closet, The (1995)]",[Documentary]
5,[Everest (1998)],[Documentary]
6,"[Brandon Teena Story, The (1998)]",[Documentary]
7,[Unzipped (1995)],[Documentary]
8,[Buena Vista Social Club (1999)],[Documentary]
9,[Heidi Fleiss: Hollywood Madam (1995)],[Documentary]


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

아래 ```send_movie_click(USER_ID, ITEM_ID)``` 는 다음과 같은 작업을 하여 실시간 이벤트를 반영 합니다.

- 유저에 대한 세션이 없으면 SessionID를 생성하고, 있으면 이 유저의 SessionID를 불러온다
- event를 정의하고 event에 item_id를 할당한다
- event를 json 형태로 바꾼다
- 위에서 생성한 event tranker의 put_event에 위 json파일을 인자로 넘겨서 호출한다.

#### Help 함수

In [270]:
session_dict = {}

def send_movie_click(user_id, ITEM_ID, verbose=False):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[user_id]
    except:
        session_dict[user_id] = str(uuid.uuid1())
        session_ID = session_dict[user_id]
   
    value=randint(0,5)
    
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    "eventValue": value
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= user_id,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'RATING',
        'properties': event_json
        }])
            
                    
    if verbose:
        print("trankingId: ", TRACKING_ID)
        print("USER_ID: ", user_id)
        print("sessionID: ", session_ID)
        print("timestamp: ", int(time.time()))
        print("properties: ", event_json)
        

    
# Help 함수

def rec_item_rich_list(item_list, clicked_title, clicked_genre):
    recommendation_title_list = []
    recommendation_genre_list = []

    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    

    title_df = pd.DataFrame(recommendation_title_list, columns=[clicked_title])
    genre_df = pd.DataFrame(recommendation_genre_list, columns=[clicked_genre])        
    recommendations_df = title_df.join(genre_df)

    return recommendations_df                                                    

def get_new_recommend(user_id,campaign_arn, recommendations_df, title, genre):    
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )

    item_list = get_recommendations_response['itemList']
    new_recs_df = rec_item_rich_list(item_list,title,  genre)
    new_recommendations_df = pd.concat([recommendations_df,new_recs_df ], axis=1)

    print("Recommendations for user: ", user_id)
    return new_recommendations_df
    

## Movie 선택 및 두번째 추천 리스트

아래를 선택 했습니다.
이후에 추천의 변화를 보시죠.<br>
movie_title: Hoop Dreams (1994), movie_genre: Documentary 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째 컬럼은 오지널 추천 영화의 제목 및 장르 입니다. <br>
세번째의 컬럼의 맨위는 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 네번째 컬럼의 맨위는 세번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [271]:
pd.options.display.max_columns = 10

import time

# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
movie_to_click = 246 # movie_title: Hoop Dreams (1994), movie_genre: Documentary
# movie_to_click = 2 # movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy
##########################

movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
print("current user_id: ", user_id)
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click, verbose=True)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
# # 새로운 추천 결과
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,movie_title_clicked, movie_genre_clicked )    

rec_exp_df


user_id: 3100, movie_id_clicked: 246, movie_title: Hoop Dreams (1994), movie_genre: Documentary
current user_id:  3100
trankingId:  50d0f462-527d-4cb4-9252-1e77c9476cf1
USER_ID:  3100
sessionID:  b8498e2c-6942-11eb-bb59-5d025f7a12e9
timestamp:  1612702084
properties:  {"itemId": "246", "eventValue": 1}
Recommendations for user:  3100


,OriginalRecs,Genre,"(Hoop Dreams (1994),)","(Documentary,)"
0,"[War Room, The (1993)]",[Documentary],"War Room, The (1993)",Documentary
1,[When We Were Kings (1996)],[Documentary],When We Were Kings (1996),Documentary
2,[Paris Is Burning (1990)],[Documentary],Paris Is Burning (1990),Documentary
3,[Wild Man Blues (1998)],[Documentary],Wild Man Blues (1998),Documentary
4,"[Celluloid Closet, The (1995)]",[Documentary],"Celluloid Closet, The (1995)",Documentary
5,[Everest (1998)],[Documentary],"Brandon Teena Story, The (1998)",Documentary
6,"[Brandon Teena Story, The (1998)]",[Documentary],Unzipped (1995),Documentary
7,[Unzipped (1995)],[Documentary],Stop Making Sense (1984),Documentary
8,[Buena Vista Social Club (1999)],[Documentary],Everest (1998),Documentary
9,[Heidi Fleiss: Hollywood Madam (1995)],[Documentary],Buena Vista Social Club (1999),Documentary


## 또 다른 Movie 선택 및 세번째 추천 리스트

그 다음에는 다큐먼트 장르의 영화를 클릭한 것을 시뮬레이션 합니다.
movie_title: War Room, The (1993), movie_genre: Documentary 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째, 세번째, 네번째는 위에서 확인한 것과 동일 합니다.<br>
다섯번째의 컬럼의 맨위는 이번에 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 여섯번째 컬럼의 맨위는 다섯번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [272]:


# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
# movie_to_click = 588 # movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical
movie_to_click = 556 # movie_title: War Room, The (1993), movie_genre: Documentary
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, 
                                                                                   movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,
                                       movie_title_clicked, movie_genre_clicked )    
rec_exp_df

user_id: 3100, movie_id_clicked: 556, movie_title: War Room, The (1993), movie_genre: Documentary
Recommendations for user:  3100


,OriginalRecs,Genre,"(Hoop Dreams (1994),)","(Documentary,)","(War Room, The (1993),)","(Documentary,)"
0,"[War Room, The (1993)]",[Documentary],"War Room, The (1993)",Documentary,When We Were Kings (1996),Documentary
1,[When We Were Kings (1996)],[Documentary],When We Were Kings (1996),Documentary,"War Room, The (1993)",Documentary
2,[Paris Is Burning (1990)],[Documentary],Paris Is Burning (1990),Documentary,Paris Is Burning (1990),Documentary
3,[Wild Man Blues (1998)],[Documentary],Wild Man Blues (1998),Documentary,Wild Man Blues (1998),Documentary
4,"[Celluloid Closet, The (1995)]",[Documentary],"Celluloid Closet, The (1995)",Documentary,"Celluloid Closet, The (1995)",Documentary
5,[Everest (1998)],[Documentary],"Brandon Teena Story, The (1998)",Documentary,Everest (1998),Documentary
6,"[Brandon Teena Story, The (1998)]",[Documentary],Unzipped (1995),Documentary,Trekkies (1997),Documentary
7,[Unzipped (1995)],[Documentary],Stop Making Sense (1984),Documentary,Stop Making Sense (1984),Documentary
8,[Buena Vista Social Club (1999)],[Documentary],Everest (1998),Documentary,"Brandon Teena Story, The (1998)",Documentary
9,[Heidi Fleiss: Hollywood Madam (1995)],[Documentary],Buena Vista Social Club (1999),Documentary,Unzipped (1995),Documentary


## 3번째 추천 영화

In [273]:
import time

# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
# movie_to_click = 1907 # movie_title: Mulan (1998), movie_genre: Animation|Children's
movie_to_click = 2677 # movie_title: Buena Vista Social Club (1999), movie_genre: Documentary
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, 
                                                                                   movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,
                                       movie_title_clicked, movie_genre_clicked )    
rec_exp_df

user_id: 3100, movie_id_clicked: 2677, movie_title: Buena Vista Social Club (1999), movie_genre: Documentary
Recommendations for user:  3100


,OriginalRecs,Genre,"(Hoop Dreams (1994),)","(Documentary,)","(War Room, The (1993),)","(Documentary,)","(Buena Vista Social Club (1999),)","(Documentary,)"
0,"[War Room, The (1993)]",[Documentary],"War Room, The (1993)",Documentary,When We Were Kings (1996),Documentary,When We Were Kings (1996),Documentary
1,[When We Were Kings (1996)],[Documentary],When We Were Kings (1996),Documentary,"War Room, The (1993)",Documentary,"War Room, The (1993)",Documentary
2,[Paris Is Burning (1990)],[Documentary],Paris Is Burning (1990),Documentary,Paris Is Burning (1990),Documentary,Paris Is Burning (1990),Documentary
3,[Wild Man Blues (1998)],[Documentary],Wild Man Blues (1998),Documentary,Wild Man Blues (1998),Documentary,Wild Man Blues (1998),Documentary
4,"[Celluloid Closet, The (1995)]",[Documentary],"Celluloid Closet, The (1995)",Documentary,"Celluloid Closet, The (1995)",Documentary,"Celluloid Closet, The (1995)",Documentary
5,[Everest (1998)],[Documentary],"Brandon Teena Story, The (1998)",Documentary,Everest (1998),Documentary,Everest (1998),Documentary
6,"[Brandon Teena Story, The (1998)]",[Documentary],Unzipped (1995),Documentary,Trekkies (1997),Documentary,"Brandon Teena Story, The (1998)",Documentary
7,[Unzipped (1995)],[Documentary],Stop Making Sense (1984),Documentary,Stop Making Sense (1984),Documentary,Unzipped (1995),Documentary
8,[Buena Vista Social Club (1999)],[Documentary],Everest (1998),Documentary,"Brandon Teena Story, The (1998)",Documentary,Buena Vista Social Club (1999),Documentary
9,[Heidi Fleiss: Hollywood Madam (1995)],[Documentary],Buena Vista Social Club (1999),Documentary,Unzipped (1995),Documentary,Thirty-Two Short Films About Glenn Gould (1993),Documentary


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다.
- 위에 유저는 기존에 Thriller, Sci-Fic 영화를 주로 봤는데요, Documentary영화를 세번 상호 작용한 후에 추천 리스트가 바뀌는 것을 확인 할 수 있었습니다.